In [284]:
import aeon
from aeon.datasets import load_from_tsf_file
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import pycatch22 as catch22
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [51]:
df, metadata = load_from_tsf_file("/Users/joshua/Documents/QuantumInspiredML/Sampling/electricity_hourly/electricity_hourly_dataset.tsf")

In [52]:
metadata

{'frequency': 'hourly',
 'forecast_horizon': None,
 'contain_missing_values': False,
 'contain_equal_length': True}

In [53]:
ts1 = df.iloc[1].series_value.to_numpy()

In [54]:
len(ts1)/24

1096.0

# How can we turn this into an interpolation problem?

Weekend versus weekday? Aggregate all households.

In [55]:
df

,series_name,start_timestamp,series_value
0,T1,2012-01-01 00:00:01,"[14.0, 18.0, 21.0, 20.0, 22.0, 20.0, 20.0, 20...."
1,T2,2012-01-01 00:00:01,"[69.0, 92.0, 96.0, 92.0, 91.0, 92.0, 91.0, 92...."
2,T3,2012-01-01 00:00:01,"[234.0, 312.0, 312.0, 312.0, 312.0, 187.0, 138..."
3,T4,2012-01-01 00:00:01,"[415.0, 556.0, 560.0, 443.0, 346.0, 340.0, 376..."
4,T5,2012-01-01 00:00:01,"[215.0, 292.0, 272.0, 213.0, 190.0, 178.0, 199..."
...,...,...,...
316,T317,2012-01-01 00:00:01,"[48.0, 65.0, 64.0, 65.0, 75.0, 64.0, 65.0, 69...."
317,T318,2012-01-01 00:00:01,"[38.0, 47.0, 43.0, 39.0, 40.0, 39.0, 49.0, 55...."
318,T319,2012-01-01 00:00:01,"[1558.0, 2177.0, 2193.0, 1315.0, 1378.0, 1250...."
319,T320,2012-01-01 00:00:01,"[182.0, 253.0, 218.0, 195.0, 191.0, 185.0, 191..."


In [196]:
df.iloc[1].start_timestamp.day_name()

'Sunday'

In [195]:
ts1 = df.iloc[1].series_value.to_numpy()

Try and window the data

In [145]:
len(ts1)/24.0

1096.0

In [211]:
windows = np.zeros(shape=(1096, 24))
labels = np.zeros(1096)
lower = 0
upper = 24
current_date = df.iloc[1].start_timestamp
for i in range(0, 1096):
    windows[i, :] = ts1[lower:upper]
    lower = upper
    upper += 24
    if (current_date.day_of_week == 5) or (current_date.day_of_week == 6):
        labels[i] = 1
    current_date += datetime.timedelta(hours=24) 

In [234]:
class_0_idxs = np.argwhere(labels == 0).flatten()
class_1_idxs = np.argwhere(labels == 1).flatten()
print(f"Num class 0: {len(class_0_idxs)} | Num class 1: {len(class_1_idxs)}") 

Num class 0: 783 | Num class 1: 313


Make an 80/20 train/test split

In [290]:
from sktime.classification.feature_based import Catch22Classifier
from sktime.classification.ensemble import Ra

In [238]:
X_train, X_test, y_train, y_test = train_test_split(windows, labels, test_size=0.2, random_state=42)

In [295]:
ss_fit = StandardScaler().fit(X_train)
X_train_scaled = ss_fit.transform(X_train)
X_test_scaled = ss_fit.transform(X_test)

In [296]:
svc_fit = SVC().fit(X_train_scaled, y_train)
svc_fit.score(X_test_scaled, y_test)

0.9136363636363637

In [314]:
#np.savetxt("electricity_y_test.txt", y_test)